In [ ]:
%%capture
!pip install uproot

In [21]:
import os

# Define your workspace directory
workspace_dir = '/content/anomaly-detection-in-HEP/'

# Create the directory if it doesn't exist
if not os.path.exists(workspace_dir):
    os.makedirs(workspace_dir)

# Change the current working directory to your workspace directory
os.chdir(workspace_dir)

# Verify the current working directory
print('Current working directory:', os.getcwd())


Current working directory: /content/anomaly-detection-in-HEP


In [22]:
from preprocessing.dataset import dataset
from torch.utils.data import DataLoader
from training.trainer import trainer
from models.vae import VAE
from models.ode import ode
from evaluation.evaluator import evaluator
import torch.optim as optim
from sklearn.model_selection import train_test_split
import json
import torch
import os
import sys
from copy import copy
from torchsummary import summary

ModuleNotFoundError: No module named 'preprocessing'

## Background data

In [ ]:
from google.cloud import storage
from io import BytesIO
import uproot

def get_data_from_root(file_dir:str, bucket_name:str = 'cuda-programming-406720'):
        client = storage.Client()
        bucket = client.get_bucket(bucket_name)
        blob = bucket.blob(file_dir)
        file_contents = BytesIO(blob.download_as_string())
        tree = uproot.open(file_contents)
        data = tree['Delphes']
        return data

data_file = 'QCD_LLP_samples/root-files/qcd_100k.root'
data = get_data_from_root(data_file)
features = ['Track.PT', 'Track.Eta', 'Track.Phi', 'Track.D0', 'Track.DZ']

In [ ]:
from itertools import chain
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

data_train = []
data_test = []
for i in range(5):
  d = data['Track'][features[i]].array().tolist()
  d = list(chain.from_iterable(d))
  data_train.append(d[:20_000])
  data_test.append(d[20_000:21_000])
data_bkg_train = torch.tensor(data_train).T
data_bkg_test = torch.tensor(data_test).T

In [ ]:
means = data_bkg_train.mean(dim=0)
stds = data_bkg_train.std(dim=0)

data_bkg_train = (data_bkg_train - means) / stds
data_bkg_test =(data_bkg_test - means) / stds
print("Train shape:", data_bkg_train.shape)
print("Test shape:", data_bkg_test.shape)

Train shape: torch.Size([20000, 5])
Test shape: torch.Size([1000, 5])


## Signal data

In [ ]:
import h5py
def get_data_from_h5(file_dir:str, bucket_name:str = 'cuda-programming-406720'):
        client = storage.Client()
        bucket = client.get_bucket(bucket_name)
        blob = bucket.blob(file_dir)
        file_contents = BytesIO(blob.download_as_string())
        with h5py.File(file_contents, 'r') as f:
            dataset = f['Track']
            data = dataset[:10_000]
            d1 = dataset[:10_000]
            d2 = dataset[10_000:10_500]
        return d1, d2

In [ ]:
sig1_train, sig1_test = get_data_from_h5('QCD_LLP_samples/h5-files/500GeV_n3_events_100k_1mm_pileup.h5')
sig2_train, sig2_test = get_data_from_h5('QCD_LLP_samples/h5-files/100GeV_n3_events_100k_1mm_pileup.h5')

In [ ]:
data_sig_train = np.concatenate((sig1_train,sig2_train))
data_sig_test = np.concatenate((sig1_test,sig2_test))

In [ ]:
data_sig_train = torch.tensor([list(data_sig_train[i]) for i in range(data_sig_train.shape[0])])
data_sig_test = torch.tensor([list(data_sig_test[i]) for i in range(data_sig_test.shape[0])])

In [ ]:
data_sig_train = (data_sig_train - means) / stds
data_sig_test =(data_sig_test - means) / stds

print("Train shape:", data_sig_train.shape)
print("Test shape:", data_sig_test.shape)

Train shape: torch.Size([20000, 5])
Test shape: torch.Size([1000, 5])


# Train VAE model

In [19]:
!python code/main.py vae

2024-05-15 22:05:42.509938: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-15 22:05:42.509992: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-15 22:05:42.511495: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-15 22:05:42.519502: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-15 22:05:43.552195: W tensorflow/comp